In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("Employee Handbook copy.pdf")
docs  = loader.load()
docs

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)


[Document(metadata={'source': 'Employee Handbook copy.pdf', 'page': 0}, page_content="                   SAMPLE EMPLOYEE HANDBOOK                            Signature:                   Print Name:                   Date:            Please sign and return to the Manager of Finance and Administration  Wisme'ac  EMPLOYEE HANDBOOK    TABLE OF CONTENTS       PAGE      I.   MISSION .....................................................................................................................................X II.   OVERVIEW.................................................................................................................................X  "),
 Document(metadata={'source': 'Employee Handbook copy.pdf', 'page': 1}, page_content="Wismettac Employee Handbook  ii ©2004 National Council of Nonproﬁt Associations  May be duplicated, with a>ribution, by charitable organizations.    III.    VOLUNTARY AT-WILL EMPLOYMENT................................................................

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
                               chunk_size = 1000,
                               chunk_overlap = 200, 
                               )

chunkk_documents = text_splitter.split_documents(docs)

chunkk_documents

[Document(metadata={'source': 'Employee Handbook copy.pdf', 'page': 0}, page_content="SAMPLE EMPLOYEE HANDBOOK                            Signature:                   Print Name:                   Date:            Please sign and return to the Manager of Finance and Administration  Wisme'ac  EMPLOYEE HANDBOOK    TABLE OF CONTENTS       PAGE      I.   MISSION .....................................................................................................................................X II.   OVERVIEW.................................................................................................................................X"),
 Document(metadata={'source': 'Employee Handbook copy.pdf', 'page': 1}, page_content="Wismettac Employee Handbook  ii ©2004 National Council of Nonproﬁt Associations  May be duplicated, with a>ribution, by charitable organizations.    III.    VOLUNTARY AT-WILL EMPLOYMENT.............................................................................X IV.  E

In [3]:
## Vector EMbeddings

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(chunkk_documents,
                          OpenAIEmbeddings()
                          )

In [ ]:
query = "working hours of an employee"
retrived_result = db.similarity_search(query)
print(retrived_result[0].page_content)

In [4]:
## Design chat prompt template 

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
I will tip you $1000 if the user finds the answer helpful.
< context>
{context} 
</ context>
Question : {input}                                         
"""                                   
)

In [6]:
from langchain_community.llms import openai
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x134756800>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1347f8940>, root_client=<openai.OpenAI object at 0x134754a90>, root_async_client=<openai.AsyncOpenAI object at 0x134756860>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [8]:
## Chain 
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context.\nThink step by step before providing a detailed answer.\nI will tip you $1000 if the user finds the answer helpful.\n< context>\n{context} \n</ context>\nQuestion : {input}                                         \n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x134756800>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1347f8940>, root_client=<openai.OpenAI object at 0x134754a90>, root_async_client=<openai.AsyncOpen

In [11]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1346a56f0>, search_kwargs={})

In [12]:
## Retriver chain
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever , document_chain)

In [15]:
response = retrieval_chain.invoke({"input" : "Work hours of an employee"})
response['answer']

"The normal work hours for an employee at Wisme'ac are from 9:00 a.m. to 5:00 p.m., Monday through Friday, including one hour (unpaid) for lunch. Employees may request to vary their work schedules within employer-defined limits to better accommodate personal responsibilities."

In [16]:
response = retrieval_chain.invoke({"input" : "To which company is this handbook referring to?"})
response['answer']

'This handbook is referring to the company Wismettac.'

In [ ]:
# user_query = st.text_input("Enter your question:", "")
# if user_query:
#     try:
#         response = retrieval_chain.invoke({"input": user_query})
#         #st.write(response)
#         st.write(response['answer'])
#     except Exception as e:
#         st.error("An error occurred: " + str(e))